<a href="https://colab.research.google.com/github/Avonna/Avona/blob/main/pickle_sec_investigation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  DataScience и безопасность: пакет pickle

### Дисклеймер

---
Материал предоставлен для ознакомления, анализа и обсуждения исключительно 
в образовательных целях. Автор не несет отвественности за использование представленного 
материала в незаконных целях третьими лицами и не подлежит судебному преследованию за его использование.
Кроме того, автор не несет ответственности за вред, возникший в результате использования 
материала. 

Код в материале предоставляется "as is", автор оставляет за собой право изменять его на
свое усмотрение и изменять условия его распространения.

---

Вобщем, все персонажи вымышлены, любое совпадение является иллюзией, игрой воображения, просто совпадением и не более того :)

### Введение

В ML достаточно часто практикуют подход сохранения и загрузки предобученных моделей 
и прочих объектов на файловую систему с помощью пакета pickle. В целом подход оправдывает 
себя, когда в условиях ограниченных вычислительных ресурсов и/или длительного времени 
обработки данных и обучения разумно фиксировать шаги процесса обучения. Даже серьезные проекты, 
например StyleGAN2, на борту несут функционал для работы с предобученными моделями с помощью пакета 
pickle (https://github.com/NVlabs/stylegan2/blob/master/pretrained_networks.py)

Тем неменее у пакета pickle есть особенность, которой не уделяют должного внимания, а именнно: 
* опасность десериализации объектов из файлов через класс Unpickler и его производные, например через методы pickle.load() или pickle.loads().

Сегодня мы поговорим о том, насколько опасен всем привычный пакет pickle и как себя
обезопасить.

**Мем в тему:**

\- Джон, у нас дыра в безопасности!

\- Слава богу, что у нас хоть что-то в безопасности!

### "Я есть pickle"

Начнем с того, что проведем небольшой ликбез по пакету pickle, что он из себя представляет 
и как его "готовят и едят". 

Допустим, вы долго трудитесь над некоторым проектом, например учите модельку, и у вас 
есть множество  шагов, которые вы не хотели бы повторять заново  в случае, если что-то 
пойдет не так. Или просто хотите зафиксировать некоторый результат, чтобы поделиться им с 
сообществом. Что делать? 

Правильно! Нужно сохранить это состояние на носителе в виде файла. А затем, в случае 
необходимости, это состояние загрузить из файла. Представим, что у нас есть 
некоторый python объект класса MyCalss и мы не хотим его "потерять":

In [ ]:
class MyClass:
    def __init__(self, *, name, number):
        self.name = name
        self.number = number
        
    def make_ops(self):
        print('make_ops() called')
        
    def __str__(self):
        return f'name: "{self.name}", number: {self.number}'
        
myobject = MyClass(name='I`m very important object! So don`t forget me!', 
                   number=1234567)

"Традиционный" опыт предшественников (а в моем случае - начинающих сайентологов) подсказывает, 
что нам следует импортировать **pickle** и вызвать метод **dump**, передав в него наш объект 
и дескриптор потока ввода-вывода, в который объект будет "записан".

In [ ]:
import pickle

with open('myobject.pkl', 'wb') as fd:
    pickle.dump(myobject, fd, 0)

Отлично, наш объект теперь в файле _myobject.pkl_. Мы легко можем его восстановить с помощью метода **pickle.load**:

In [ ]:
myobject = None

with open('myobject.pkl', 'rb') as fd:
    myobject = pickle.load(fd)

print(f'Object type stored within the myobject.pkl: \n\t{type(myobject)}')
print(f'myobject.name: \n\t{myobject.name}')
print(f'myobject.number: \n\t{myobject.number}')
myobject.make_ops()

Object type stored within the myobject.pkl: 
	<class '__main__.MyClass'>
myobject.name: 
	I`m very important object! So don`t forget me!
myobject.number: 
	1234567
make_ops() called


Как видим информация о типе объекта (классе), переменных и их значениях не утеряна 
после восстановления объекта из файла, что очень здорово! Мы можем возобновить работу 
с объектом в любом месте нашего проекта!

Давайте посмотрим, что внутри нашего файла. Как выглядит упакованный объект? 

In [ ]:
with open('myobject.pkl','rb') as fd:
    serialized_data = fd.read()
    print(serialized_data)

b'ccopy_reg\n_reconstructor\np0\n(c__main__\nMyClass\np1\nc__builtin__\nobject\np2\nNtp3\nRp4\n(dp5\nVname\np6\nVI`m very important object! So don`t forget me!\np7\nsVnumber\np8\nI1234567\nsb.'


Какая-то абракадабра. 

Можем взглянуть под другим углом и получить более осмысленную картину, 
для чего будем использовать пакет _pickletools_:

In [ ]:
import pickletools as pt

with open('myobject.pkl','rb') as fd:
    serialized_data = fd.read()
    print( pt.dis(serialized_data) )

    0: c    GLOBAL     'copy_reg _reconstructor'
   25: p    PUT        0
   28: (    MARK
   29: c        GLOBAL     '__main__ MyClass'
   47: p        PUT        1
   50: c        GLOBAL     '__builtin__ object'
   70: p        PUT        2
   73: N        NONE
   74: t        TUPLE      (MARK at 28)
   75: p    PUT        3
   78: R    REDUCE
   79: p    PUT        4
   82: (    MARK
   83: d        DICT       (MARK at 82)
   84: p    PUT        5
   87: V    UNICODE    'name'
   93: p    PUT        6
   96: V    UNICODE    'I`m very important object! So don`t forget me!'
  144: p    PUT        7
  147: s    SETITEM
  148: V    UNICODE    'number'
  156: p    PUT        8
  159: I    INT        1234567
  168: s    SETITEM
  169: b    BUILD
  170: .    STOP
highest protocol among opcodes = 0
None


Файл _myobject.pkl_ менее информативен, поскольку протоколы сериализации/десериализации 
не в последнюю очередь призваны для минимизации занимаего дискового пространства при работе с объектом 
на файловой системе и сохранения топологии объекта. В файле есть структура, и из листинга мы 
можем её наблюдать. Файл состоит из записей, каждую из которых предваряет токен и значение, каждому 
токену соотвествует его символьное придставление, которое лежит в файле. Например, на позиции 25 
начинается запись с токеном PUT, которому соотвествует символ _p_ в файле. Само значение может 
быть простым или составным (то есть контейнером типа MARK). Стоит отметить, что прежде 
всего в файле сохранены такие параметры как:

* пространство \_\_main\_\_ в котором определен атрибут (тип сериализованного инстанса)
* имя атрибута (типа _MyClass_ )
* члены-переменные ( _name_ и _number_ ) с их значениями

При этом в файле нет информации о членах-методах класса, конструкторе и прочих вещах, 
не говоря об их реализации. Стоит отметить, что при десериализации нашего инстанса из файла
класс, указанный в файле, должен быть объявлен в модуле, иначе в процессе работы load/loads
будет выброшено исключение типа 

_AttributeError: Can't get attribute 'MyClass' on <module '__main__'>_

Что же происходит в недрах пакета pickle? Согласно документации пакет pickle реализует 
несколько версий протоколов сериализации python объектов в двоичный поток (в файл) и их 
десериализации из "сырых" данных. Если вы знакомы с технологиями COM, RPC, Java Searializable, 
то здесь очень четкая аналогия: "упаковка" (сериализация) объекта в двоичный формат и 
распаковка (десериализация). В контексте пакета pickle терминология следущая: 
* "pickling" - это сериализация, упаковка, маршалинг, безопасный хомячок.
* "unpickling" - это десериализация, распаковка, котик, который может сделать "кусь"


В недрах pickle за сериализацию объекта отвечает класс Pickler, за десериализацию - Unpickler.


Что может быть сериализовано? Ответ нам дает документация:
* None, True, and False;
* integers, floating-point numbers, complex numbers;
* strings, bytes, bytearrays;
* tuples, lists, sets, and dictionaries containing only picklable objects;
* functions (built-in and user-defined) accessible from the top level of a module (using def, not lambda);
* classes accessible from the top level of a module;
* instances of such classes whose the result of calling __getstate__() is picklable (see section Pickling Class Instances for details).

Странно, что здесь заявлена возможность сериализации функций. Ведь в примерах 
выше мы видели, что тело функций не сериализуется, только имя функции, объявленное
в глобальном пространстве. Возможно, у автора кривые руки и вы поправите его по этому 
выводу.

Кроме того, разработчики с заботой предупреждают нас, что пакет pickle явлется небезопасным (https://docs.python.org/3/library/pickle.html):

---

**[Warning The pickle module is not secure. Only unpickle data you trust.
It is possible to construct malicious pickle data which will execute arbitrary code during unpickling. Never unpickle data that could have come from an untrusted source, or that could have been tampered with.
Consider signing data with hmac if you need to ensure that it has not been tampered with.
Safer serialization formats such as json may be more appropriate if you are processing untrusted data. See Comparison with json.]**

---


### Что не так с Unpickler?

Вспомните пример выше, когда мы с помощью pickletools вывели в осмысленном виде содержимое 
нашего файла, содержащего сериализованный инстанс. Он нам демонстрирует записи с токеном GLOBAL. 
Взгялните еще раз на этот пример, записи с токенами GLOBAL на позициях 0, 29 и 50.
В файле целых 3 записи: 
* 1-я определяет, что к инстансу нужно применить метод \_reconstructor из пространства copy_reg
* 2-я определяет, что инстанс имеет тип "класс MyClass", который должен быть объявлен в пространстве (модуле) \_\_main\_\_; 
* 3-я определяет, что инстанс - это объект(?), который нужно создать методом _object()_ из пространства (модуля) \_\_builtin\_\_. 

А что же за пространства copy_reg и \_\_builtin\_\_? Вспоминаем азы python-а, в частности 
обратимся к разделу документации https://docs.python.org/3/library/builtins.html. В нем нам рассказывают
о встроенных объектах языка: функциях и константах. То есть слово \_\_builtin\_\_ из файла соотвествует 
модулю builtins, а слово object из файла соотвествует встроенной функции object() в модуле builtins. 
В примере выше модули copy_reg и \_\_builtin\_\_ соотвествуют версии Python2.x, что вобщем-то 
не удивительно: пакет pickle придумали на заре становления языка, поэтому это - "эхо далекой войны", 
так сказать. И сделано в угоду совместимости файлов, созданных в версиях Python2.x, на более свежих версиях интерпретатора. Почему же это работает на версиях Python3.x? Магия кроется в файле _compat_pickle.py, который содержит маппинг старых имен модулей и функций (типа xrange -> range) из Python2.x в новые для Python3.x:



```
# This module is used to map the old Python 2 names to the new names used in
# Python 3 for the pickle module.  This needed to make pickle streams
# generated with Python 2 loadable by Python 3.

# This is a copy of lib2to3.fixes.fix_imports.MAPPING.  We cannot import
# lib2to3 and use the mapping defined there, because lib2to3 uses pickle.
# Thus, this could cause the module to be imported recursively.
IMPORT_MAPPING = {
    '__builtin__' : 'builtins',
    'copy_reg': 'copyreg',
    'Queue': 'queue',
    'SocketServer': 'socketserver',
    'ConfigParser': 'configparser',
    'repr': 'reprlib',
    ...
```



**Хм, вроде ничего страшного на первый взгляд.**

Но в модуле _builtins_ наряду с функцией _object()_ присутствует ряд других функций, 
куда менее безобидных. Например, _eval()_, _compile()_ или _exec()_. 

Ниже пример для затравки (спасибо разработчикам за любезно предоставленый код), который 
десериализует инстанс, в котором применяется встроенная в язык функция eval(), и в целом 
демонстрирует неожиданный (для тех, кто не читает man-ы !) эффект процесса десериализации. 
В примере выполняется тело _'print(1 + 2)'_ с помощью функции eval():

In [ ]:
pickle.loads((b"c__builtin__\neval\n(S'print(1 + 2)'\ntR."))

3


Да, именно об этом эффекте нас предупреждают разработчики в своей документации. Палец вверх если не знали или пропустили мимо ушей при освоении документации!

В примере выше использован метод _loads_. Он отличается от _load_ тем, что на вход принимает _bytes-like_ объект в место потока ввода-вывода. Концептульно разницы никакой: load и loads являются обертками на классом _Unpickler_.

Как вы могли заметить, метод load успешно "распаковал" наш инстанс (точнее, разобрал двойчные данные согласно протокола): никаких ошибок в процессе десериализации не возникло. Также он "вывел в консоль" чиселку, являющуюся суммой 1 и 2. Необычно, но пока ничего серьезного, не так ли?

**Уже не так безобидно!**

А что еще такого можно придумать? Как мы увидели выше, запись GLOBAL имеет спецификацию:
GLOBAL MODULE_NAME ATTRIBUTE_NAME. Сделаем предположение о том, что мы можем указать произвольный модуль,
например _os_ и дернуть из него метод или свойство, например _os.name_:

In [ ]:
pickle.loads(b"cos\nname\n.") # не ставим ';'

'posix'

Ух!! Работает! Повышаем ставки: посмотрим имя текущего системного пользователя, используя пакет os. Будем дергать метод os.system(), который позволяет выполнять системные команды и запускать произвольные программы:

In [ ]:
# pickle.loads(b"cos\nsystem\n(S'id -un > usr.txt'\ntR.") # for Mac OS
status = pickle.loads(b"cos\nsystem\n(S'whoami > whoami_result.txt'\ntR.")
print(f'whoami status code: {status}')
with open('whoami_result.txt', 'rb') as fd:
  print(f'current user: {fd.read().decode()}')

whoami status code: 0
current user: root



**Хм, такого поведения вы точно не ожидали и не желали для себя!**

Давайте попробуем что-нибудь по-интересней, например посмотреть наш текущий каталог с помощью команды pwd (согласен, пример притянут за уши, но нам важен смысл, а не обертка):

In [ ]:
status = pickle.loads(
    b"cos\nsystem\n(S'pwd > pwd_status.txt'\ntR."
    );
print(f'pwd status code: {status}')
with open('pwd_status.txt', 'rb') as fd:
  print(f'current directory: {fd.read().decode()}')

pwd status code: 0
current directory: /content



Как видим, в процедуре unpickling спрятано гораздо больше, чем кажется. Безобидная 
на первый взгляд зверушка может стать монстром и доставить немало хлопот. 

**Простыми
словами:** через десериализацию с помощью пакета pickle из коробки вы подвергаете себя
риску.

### Игра в иммитацию

Давайте смоделируем ситуацию, когда вы получили из какого-либо источника сериализованный 
объект. Например, клонировали с помощью git файлы предобученных моделей (как в примере выше 
с github/NVlabs компания NVidia любезно разместила файлы предобученных моделей на своих серверах, 
а в git залила кодовую базу). Для примера, пусть это будет файл _pretrained_model_0.pkl_. Вы обычно 
не задумываетесь о его безопасности (надеюсь, что после прочтения этой публикации вы будете думать
по-другому) и рассматриваете его как просто файл с данными, в котором "лежит" модель. Мы смоделируем злоумышленника, который создал такой файл модели и предоставил его вам. В файл он поместил что-то вредоносное: в нашем примере выполнение команды ls -l > ls_status.txt и вывод результата в "консоль":

In [ ]:
script = b'import os; os.system("ls -l > ls_status.txt");\r' + \
         b'with open("ls_status.txt", "rb") as fd: print(fd.read().decode())'
with open('pretrained_model_0.pkl', 'wb') as fd:
    fd.write(b"cbuiltins\nexec\n(S'" + script + b"'\ntR.")

И так, вы уверены в безопасности кода в вашем проекте (ведь он написан и выверен вами!), и вы пытаетесь загрузить любезно предоставленную кем-то модель из файла _pretrained_model_0.pkl_ с помощью _pickle.load()_. Давайте попробуем это сделать:

In [ ]:
model = None
with open('pretrained_model_0.pkl', 'rb') as fd:
    model = pickle.load(fd)  
# you code here

total 28
-rw-r--r-- 1 root root    0 Dec 18 16:01 ls_status.txt
-rw-r--r-- 1 root root  171 Dec 18 15:43 myobject.pkl
-rw-r--r-- 1 root root    9 Dec 18 15:50 ping_status.txt
-rw-r--r-- 1 root root  135 Dec 18 16:01 pretrained_model_0.pkl
-rw-r--r-- 1 root root    9 Dec 18 15:53 pwd_status.txt
drwxr-xr-x 1 root root 4096 Dec 16 00:01 sample_data
-rw-r--r-- 1 root root    5 Dec 18 15:48 usr.txt
-rw-r--r-- 1 root root    5 Dec 18 15:53 whoami_result.txt



В обычном сценарии вы получили бы объект модели в переменной _model_. 

Но, кто-то выбрал вас, и в худшем варианте, вы стали полноценным участником ботнета или очередной жертвой ransomware, что демонстрирует нам вывод команды ls -l. Ведь вместо безобидного вызова ls -l ваш "доброжелатель"
может поместить что-то менее безобидное. Дальше можете фантазировать на предмет кибершпионажа в 
области высоких технологий и научно-технического прогресса.

### План 'B': бежим по кругу и кричим А-А-А?

К счастью разработчики pickle все же подумали о нас и предоставили нам возможность контролировать 
процессы сериализации и десериализации, что требует определенных усилий от нас как
пользователей пакета pickle. В контексте повествования мы не будем рассматривать кастомизацию процесса 
сериализации (pickling), поскольку для нас это процесс угрозы не представляет.

Что же касается процесса десериализации, то крайне рекомендуется создать свою обертку над классом
Unpickler и определить метод find_class. Суть метода find_class() - фильтровать пакеты и имена, 
заявленныe в токенах GLOBAL, на основе белых списков и выбрасывать исключения в случае, если
обнаружены неразрешенные пакет и/или аттрибут из пакета (свойство или функция).

Ниже представлен скелет, который можно использовать и дополнять под свои нуджы для безопасной десериализации объектов с помощью пакета pickle. Класс частично обеспечивает поддержку  пакетов из Python 2.x при работе в Python 3.x.

In [ ]:
import builtins
import io
import os.path
import importlib
import pickle
compat_pickle = importlib.import_module('_compat_pickle')


class SafeUnpickler(pickle.Unpickler):
    
    def __init__(self, file, *, fix_imports=True, encoding='ASCII', 
                 errors='strict', buffers=None, safe_modules=None, 
                 safe_names=None):
        
        super().__init__(file, fix_imports=fix_imports, encoding=encoding, errors=errors)
        
        # create dict of safe modules with their members using Python 2.x names
        self.safe_modules = {
            '__builtin__': [
                'xrange','complex','set','frozenset','slice','list', 'tuple', 'object'
            ],
            'copy_reg': [
                '_reconstructor',
            ],
        }
        
        # Convert Python2.x to Python3.x names
        modules = set(self.safe_modules.keys())
        
        for module in modules:
            if module in compat_pickle.IMPORT_MAPPING.keys():
                if not compat_pickle.IMPORT_MAPPING[module] in self.safe_modules:
                    self.safe_modules[ compat_pickle.IMPORT_MAPPING[module] ] = []
                    
                for name in self.safe_modules[module]:
                    t = (module, name, ) 
                    if t in compat_pickle.NAME_MAPPING.keys():
                        self.safe_modules[ compat_pickle.IMPORT_MAPPING[module] ].append(
                            compat_pickle.NAME_MAPPING[t][1]
                        )
                    else:
                        self.safe_modules[ compat_pickle.IMPORT_MAPPING[module] ].append(name)
        
        # Add additional modules and members like __main__ and etc
        #self.safe_modules[__name__] = list(__import__(__name__).__dict__.keys())
        #if isinstance(safe_names, list):
        #    for name in safe_names:
        #        self.safe_modules[__name__].append(name)
        
        if isinstance(safe_modules, dict):
            for module in safe_modules:
                if not module in self.safe_modules:
                    self.safe_modules[module] = safe_modules[module].copy()

                    
    def find_class(self, module, name):
        if module in self.safe_modules and name in self.safe_modules[module]:
            if module in compat_pickle.IMPORT_MAPPING.keys():
                names = self.safe_modules[compat_pickle.IMPORT_MAPPING[module]]
                if name in names:
                    return getattr(__import__(compat_pickle.IMPORT_MAPPING[module]), name)
            
            elif module == __name__:
                return getattr(__import__(__name__), name)
            
            else:
                names = self.safe_modules[module]
                if name in names:
                    return getattr(__import__(module), name)
        
        raise pickle.UnpicklingError("global '%s.%s' is forbidden" %
                                     (module, name))


def load(inst_type, *, filename=None, data=None):
    """
    Function loads pickled object for data source like file or binary data
    
    @type inst_type: object
    @param inst_type: specifies object type to be unpickled (requied); 
     
    @type filename: str
    @param inst_type: specifies path to pickle file (optional);
    
    @type data: bytes
    @param inst_type: specifies pickled data (optional).
    
    @rtype: object
    @return: returns unpickled object
    """
    if filename is None and data is None:
        raise Exception('Invalid arguments')
        
    obj = None
    if filename and isinstance(filename, str) and os.path.isfile(filename):
        with open(filename, 'rb') as fd:
            obj = SafeUnpickler(io.BytesIO(fd.read()),
                                safe_modules={__name__: [inst_type.__name__]} 
                               ) .load()
            
    elif data and isinstance(data, bytes):
        obj = SafeUnpickler(io.BytesIO(data),
                            safe_modules={__name__: [inst_type.__name__]} 
                           ) .load()
    else:
        raise Exception(f'Invalid data type {type(filename)}')
    return obj


def save(obj, filename):
    with open(filename, 'wb') as fd:
        p = pickle.Pickler(fd)
        p.dump(obj)

Из коробки наш SafeUnpickler поддерживает в качестве безопасных атрибутов (свойств и методов):
* 'xrange'/'range','complex','set','frozenset','slice','list', 'tuple', 'object' из \_\_builtin\_\_ (Python 2.x) и builtins (Python 3.x)
* '_reconstructor' из copy_reg (Python 2.x) и copyreg (Python 3.x)
* явно указанный тип из \_\_name\_\_

Ниже пытаемся загрузить наш инстанс (объект типа MyClass) из файла, созданного ранее, с помощью безопасного Unpickler-а:

In [ ]:
m = load(MyClass, filename='myobject.pkl')
print(str(m))

name: "I`m very important object! So don`t forget me!", number: 1234567


Теперь попробуем загрузить "заряженый" через экран телевизора файл (вспоминаем игру в иммитацию). Ожидаемо, файл отвержен по причине того, что в нем встретился неразрешенный метод exec в составе пакета builtins 

In [ ]:
p = load(MyClass, filename='pretrained_model_0.pkl')

UnpicklingError: ignored

Вобщем, наша безопасная десериализация SafeUnpickler работает. Класс не претендует на полноту, и конечно требует доработки.

### Рекомендации

* Не используйте десериализацию pickle из коробки (методы load/loads), если вы не уверены в надежности источника файлов и их целостности
* Проверяйте источник файлов данных прежде чем десериализовывать инстансы из этих файлов
* Используйте дополнительные методы, такие как цифровая подпись и хэш-суммы для файлов
* Применяйте концепцию безопасной десериализации при помощи кастомизации Unpickler
* Используйте альтернативные пакеты, такие как JSON, которые являются более безопасными

### Итоги

Мы рассмотрели:

* типовой сценарий использования пакета pickle для сериализации и десериализации
* угрозу, которая таится в механизме десериализации в пакете pickle
* способ безопасной десериализации
* рекомендации по работе

Содзано специально для:
* "Комьюнити ЦНО", 
* "First steps in NLP Bootcamp" 
* "DS-9 (2МО-9) Машинное обучение" / 2022

По вопросам и предложениям пишите: 
* на почту mr.arch.lev@gmail.com или в групповые чаты

Всем удачи и котиков!